In [3]:
import numpy as np
import dill
import importlib
import os
import matplotlib.pyplot as plt
import data_visualization as dv
import copy
import numba as nb
import time

import src.read_data_functions as rdf
import src.plot_functions as pf
import src.Optimal_Stopping_Object as oso
import src.Subject_Object_v2

%load_ext autoreload
%aimport Optimal_Stopping_Object
%autoreload 1
'''
v4 Uses the object imported from the py file class which is very similar to the Optimal_Stopping_Function_v3

As of 1/13/23, the only change is the addition of the decision to action delay
'''

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


'\nv4 Uses the object imported from the py file class which is very similar to the Optimal_Stopping_Function_v3\n\nAs of 1/13/23, the only change is the addition of the decision to action delay\n'

In [4]:
s = slice(10,50)
a = np.ones(100)
a[s]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1.])

# Select Exp1 or Exp2


In [5]:
experiment = "Exp1"

# Initial Thangs


In [6]:
plt.style.use("cashaback_dark")
wheel = dv.ColorWheel()

# Initial thangs
os.chdir(f"D:\OneDrive - University of Delaware - o365\Subject_Data\MatchPennies_Agent_{experiment}")
PATH = os.getcwd()
SAVE_PATH = f"D:\\OneDrive - University of Delaware - o365\\Subject_Data\\MatchPennies_Agent_{experiment}\\Figures\\"
if not os.path.exists(SAVE_PATH):
    os.makedirs(SAVE_PATH)

# Fields pull and pull list
figures_pull_list = []
figures_pull_list_control = []
figures_pull_list_task = []
fields_pull = []
with open(PATH + "\\Figures_Pull_List.txt", "r") as pull_file:
    figures_pull_list = pull_file.read().splitlines()
with open(PATH + "\\Fields_Pull.txt", "r") as fields_pull:
    fields_pull = fields_pull.read().splitlines()
num_subjects = len(figures_pull_list)
if experiment == "Exp2":
    tp3_title = "Win = 1\nIncorrect = 0\nIndecision = 0"
    tp4_title = "Win = 1\nIncorrect = -1\nIndecision = 0"
    tp5_title = "Win = 1\nIncorrect = 0\nIndecision = -1"
    tp6_title = "Win = 1\nIncorrect = -1\nIndecision = -1"
    trial_block_titles = [tp3_title, tp4_title, tp5_title, tp6_title]
    num_blocks = len(trial_block_titles)
    xlabel = "Payoff Condition"
if experiment == "Exp1":
    tp3_title = "1000 (50)"
    tp4_title = "1000 (150)"
    tp5_title = "1100 (50)"
    tp6_title = "1100 (150)"
    tp7_title = "1200 (50)"
    tp8_title = "1200 (150)"
    trial_block_titles = [tp3_title, tp4_title, tp5_title, tp6_title, tp7_title, tp8_title]
    num_blocks = len(trial_block_titles)
    xlabel = "Mean [SD] Agent Decision Time (ms)"
num_subjects

20

# Pickle Data


In [7]:
if "subject_objects" not in locals():
    subject_objects = rdf.generate_subject_objects(experiment)
    subject_objects2 = copy.deepcopy(subject_objects)
    # group = Subject_Object_v2.Group(subject_objects, select_trials = 'All Trials', num_stds_for_reaction_time = 2,
    #                                     task_leave_time_metric_name = 'player_pos_task_leave_time', task_movement_time_metric_name = 'player_pos_task_movement_time',
    #                                     reaction_time_metric_name = 'player_pos_reaction_time', reaction_movement_time_metric_name = 'player_pos_reaction_movement_time')
    # group.analyze_data()
    group = Subject_Object_v2.Group(
        subject_objects2,
        select_trials="All Trials",
        num_stds_for_reaction_time=2,
        task_leave_time_metric_name="player_velocity_task_leave_time_thresh",
        task_movement_time_metric_name="player_velocity_task_movement_time_thresh",
        reaction_time_metric_name="player_velocity_reaction_time_thresh",
        reaction_movement_time_metric_name="player_velocity_reaction_movement_time_thresh",
    )
    group.analyze_data()

Sub1
Sub2
Sub3
Sub4
Sub5
Sub6
Sub7
Sub8
Sub9
Sub10
Sub11
Sub12
Sub13
Sub14
Sub15
Sub16
Sub17
Sub18
Sub19
Sub20


D:\OneDrive - University of Delaware - o365\Desktop\MatchPennies-Agent-Expirement\src\Subject_Object_v2.py:557: RuntimeWarning: divide by zero encountered in true_divide
  self.mu_s = (self.phat_correct*self.mhat_correct - self.phat_error*self.mhat_error)/(self.phat_correct - self.phat_error)
D:\OneDrive - University of Delaware - o365\Desktop\MatchPennies-Agent-Expirement\src\Subject_Object_v2.py:568: RuntimeWarning: divide by zero encountered in true_divide
  self.mu_s_alternate = (self.phat_correct*self.mhat_correct_alternate - self.phat_error*self.mhat_error_alternate)/(self.phat_correct - self.phat_error)


# Run fit sequence for each individual  

In [8]:
if experiment == "Exp1":
    rt = np.nanmedian(group.reaction_time, axis=1) - 15
    rt_sd = np.nanstd(group.reaction_time, axis=1)
elif experiment == "Exp2":
    rt    = np.nanmedian(group.react_reaction_time_only_react, axis=1) - 30
    rt_sd = np.nanmedian(np.nanstd(group.reaction_time, axis=2))

mt                   = np.min(np.nanmedian(group.player_task_movement_time, axis=2), axis=1)  # Get movement time for the condition where they tried the hardest
mt_sd                = np.median(np.nanstd(group.player_task_movement_time, axis=2), axis=1)
time_sd              = np.array([(np.nanstd(group.coincidence_reach_time, axis=1))] * num_blocks).T
perc_wins_both_reach = group.perc_gamble_wins_when_both_decide
gamble_sd            = np.nanstd(group.player_gamble_task_leave_time, axis=2)
agent_sds            = np.nanstd(group.agent_task_leave_time, axis=2)
agent_means          = np.nanmean(group.agent_task_leave_time, axis=2)

model_expected_list = {}
model_true_list = {}
for i in range(1):
    subname = figures_pull_list[i]
    # model_expected = oso.ModelConstructor(
    #     experiment=experiment,
    #     num_blocks=num_blocks,
    #     num_timesteps=1800,
    #     BETA_ON=False,
    #     agent_means=agent_means[i, :],
    #     agent_sds=agent_sds[i, :],
    #     reaction_time={"true": rt[i], "exp": rt[i]},
    #     movement_time={"true": mt[i], "exp": mt[i]},
    #     reaction_sd={"true": rt_sd[i], "exp": rt_sd[i]},
    #     movement_sd={"true": mt_sd[i], "exp": mt_sd[i]},
    #     timing_sd={"true": time_sd[i], "exp": time_sd[i]},
    #     perc_wins_when_both_reach=perc_wins_both_reach[i],
    #     gamble_delay_known=True,
    #     gamble_sd_known=True,
    #     gamble_decision_sd={"true": gamble_sd[i], "exp": 10},
    #     gamble_delay={"true": 125, "exp": 50},
    #     expected=True,
    # )

    model_true = oso.ModelConstructor(
        experiment=experiment,
        num_blocks=num_blocks,
        num_timesteps=1800,
        BETA_ON=False,
        agent_means=agent_means[i, :],
        agent_sds=agent_sds[i, :],
        reaction_time={"true": rt[i], "exp": rt[i]},
        movement_time={"true": mt[i], "exp": mt[i]},
        reaction_sd={"true": rt_sd[i], "exp": rt_sd[i]},
        movement_sd={"true": mt_sd[i], "exp": mt_sd[i]},
        timing_sd={"true": time_sd[i], "exp": time_sd[i]},
        perc_wins_when_both_reach=perc_wins_both_reach[i],
        gamble_delay_known=True,
        gamble_sd_known=True,
        gamble_decision_sd={"true": gamble_sd[i], "exp": 10},
        gamble_delay={"true": 125, "exp": 50},
        expected=False,
        data_leave_times=np.nanmedian(group.player_task_leave_time, axis=2)[i],
    )

    # Fit true model
    free_params_init = {
                        'reaction_time':model_true.inputs.reaction_time['true'],
                        }
    # get_true_metric = model_true.results.get_metric
    # init_decision_time = np.array([900]*num_blocks)
    # init_model_leave_time = get_true_metric(model_true.player_behavior.wtd_leave_target_time, metric_type="optimal")
    # init_model_wins       = get_true_metric(model_true.score_metrics.prob_win, metric_type="optimal")
    # init_model_incorrects       = get_true_metric(model_true.score_metrics.prob_incorrect, metric_type="optimal")
    # init_model_indecisions       = get_true_metric(model_true.score_metrics.prob_indecision, metric_type="optimal")

    # init_model_metrics = np.array([init_decision_time,init_model_leave_time,init_model_wins,init_model_incorrects,init_model_indecisions])

    targets = np.array(
        [np.nanmedian(group.player_task_leave_time, axis=2)[i], 
        group.player_perc_wins[i], 
        group.player_perc_incorrects[i], 
        group.player_perc_indecisions[i]]
    )
    metric_keys = ['wtd_leave_target_time','prob_win','prob_incorrect','prob_indecision']
    fit_params,out = model_true.fit_multiple_parameters(
            free_params_init=free_params_init,
            targets=targets,
            metric_keys=metric_keys,
        )
    # Fit true model the old way
    # model_true.fit_model(
    #     model_true.player_behavior.wtd_leave_target_time,
    #     np.nanmedian(group.player_task_leave_time, axis=2)[i],
    # )
    # Update dicts
    model_true_list.update({subname: model_true})
    # model_expected_list.update({subname: model_expected})

AttributeError: 'ModelConstructor' object has no attribute 'initial_shape'

In [ ]:

#TODO figure out this weirdness
model_true.results.get_metric(model_true.player_behavior.wtd_leave_target_time,metric_type='fit')

array([1149.45877498, 1166.5       , 1214.87571813, 1120.19065671,
       1112.42926777, 1220.20788839])

In [ ]:
np.nanmedian(group.player_task_leave_time,axis=2)[i]

array([1149.5, 1166.5, 1215. , 1120. , 1112.5, 1220. ])

In [ ]:
model_true.results.get_metric(model_true.score_metrics.prob_incorrect,metric_type='fit')*100 - group.player_perc_incorrects[i]

array([-10.2343084 ,   2.96155529,  -2.9520296 ,  -9.03979225,
        -9.15985113,  -3.42491119])

In [ ]:
model_true.results.get_metric(model_true.score_metrics.prob_indecision,metric_type='fit')*100 - group.player_perc_indecisions[i]

array([ 1.26133779, -4.80319278,  9.17810366,  2.92205253,  5.76777487,
        8.02916781])

In [ ]:
out

 final_simplex: (array([[213.40494526],
       [213.40494526]]), array([594.63478422, 594.63478422]))
           fun: 594.6347842188381
       message: 'Optimization terminated successfully.'
          nfev: 76
           nit: 36
        status: 0
       success: True
             x: array([213.40494526])